In [1]:
from gqimax.mapper import cuda_map_cx
import cupy as cp
%load_ext autoreload
%autoreload 2
def map_cx(lambdass, indicess, control, target):
    """
    --- First, I encode the n-qubit Pauli word as list of n int8 array
    Ex: XYI --> [1, 2, 0]
    For n-stabilizer, we have n x k words, so the encoded tensor will be n x k x n (ragged tensor)
    --- Next step, I flatten this tensor to 1D array (each element is still n-dim array)
    Flatten vector: [[0 1 2]
    [0 2 2]
    [0 3 2]
    [1 2 3]
    [2 3 1]
    [2 2 1]]
    --- Map this array to the new array using map_cx kernel
    Mapped flatten vector: [[0 1 2]
    [3 2 2]
    [3 3 2]
    [2 3 3]
    [1 2 1]
    [1 3 1]]
    Lambdas: [ 1  1  1  1  1 -1]
    
    --- Finally, I unflatten the mapped array to the original shape (ragged tensor)
    --- Obviously, this function requires starts variable (the start index of each row in the flatten vector)
    
    Out ragged tensor (with starts = [0,3,4]): [array([[0, 1, 2],
        [3, 2, 2],
        [3, 3, 2]], dtype=int8), array([[2, 3, 3]], dtype=int8), array([[1, 2, 1],
        [1, 3, 1]], dtype=int8)]

    """

    def flatten_ragged_matrix_cupy(ragged_matrix):
        lengths = cp.array([len(row) for row in ragged_matrix], dtype=cp.int8)
        starts = cp.concatenate((cp.array([0]), cp.cumsum(lengths)), dtype=cp.int8)
        flatten_vector = cp.concatenate(ragged_matrix, dtype=cp.int8)
        return flatten_vector, starts[:-1]

    flatten_indicess, starts = flatten_ragged_matrix_cupy(indicess)

    flatten_lambdas_sign, mapped_flatten_indicess = cuda_map_cx(flatten_indicess, control, target)
    print("mapped_flatten_indicess", mapped_flatten_indicess)
    print("flatten_lambdas_sign", flatten_lambdas_sign)
    
    starts = starts[1:].tolist()
	# Convert flatten vector to ragged tensor
    ragged_indicess = cp.vsplit(mapped_flatten_indicess, starts)
    ragged_lambdas_sign = cp.split(flatten_lambdas_sign, starts)
	# OP: lambdas_sign * ragged_lambdas
    # This operator can be implemented in CUDA kernel (in file notebook)
    # But I see there is no different between two methods
    print(ragged_lambdas_sign)
    print(lambdass)
    return [cp.multiply(m1, m2) for m1, m2 in zip(lambdass, ragged_lambdas_sign)], ragged_indicess



In [3]:
import cupy as cp

lambdass = [
	cp.array([ 9.65925820e-01,  2.58819093e-01, -2.18556924e-08]), 
 	cp.array([1.])
]

indicess = [
    cp.array([[1, 1],[2, 1],[3, 0]]), 
    cp.array([[3, 3]], dtype = cp.int8)
]
map_cx(lambdass, indicess, 0, 1)

# [0.9659]*xi + [0.2588]*yi
# [1]*iz

mapped_flatten_indicess [[1 0]
 [2 0]
 [3 0]
 [0 3]]
flatten_lambdas_sign [1 1 1 1]
[array([1, 1, 1], dtype=int8), array([1], dtype=int8)]
[array([ 9.65925820e-01,  2.58819093e-01, -2.18556924e-08]), array([1.])]


([array([ 9.65925820e-01,  2.58819093e-01, -2.18556924e-08]), array([1.])],
 [array([[1, 0],
         [2, 0],
         [3, 0]], dtype=int8),
  array([[0, 3]], dtype=int8)])